# Importing Libraries

In [151]:
import pandas as pd
import numpy as np

# PDF Text extracttion
import tabula
import re

# Matching
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [107]:
data = pd.read_csv('flats_berlin.csv')

# Data Check

### Check for duplicated rows

In [108]:
data[data.duplicated()]

,Titel,Url,Kaltmiete,Nebenkosten,Heizkosten,Gesamtmiete,Kaution o. Genossenschaftsanteile,Etage,Wohnfläche ca.,Zimmer,...,Baujahr,Modernisierung/ Sanierung,Objektzustand,Heizungsart,Wesentliche Energieträger,Energieausweis,Energieausweistyp,Energieverbrauchskennwert,Anbieter,Adreese


### Check percentage of missing values

In [109]:
data.isna().mean()

Titel                                0.000000
Url                                  0.000000
Kaltmiete                            0.000000
Nebenkosten                          0.000000
Heizkosten                           0.000000
Gesamtmiete                          0.000000
Kaution o. Genossenschaftsanteile    0.073649
Etage                                0.105405
Wohnfläche ca.                       0.000000
Zimmer                               0.000000
Schlafzimmer                         0.427365
Badezimmer                           0.271959
Personenaufzug                       0.476014
Einbauküche                          0.359797
Baujahr                              0.121959
Modernisierung/ Sanierung            0.698649
Objektzustand                        0.226014
Heizungsart                          0.151351
Wesentliche Energieträger            0.232770
Energieausweis                       0.284797
Energieausweistyp                    0.314189
Energieverbrauchskennwert         

In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2960 entries, 0 to 2959
Data columns (total 24 columns):
Titel                                2960 non-null object
Url                                  2960 non-null object
Kaltmiete                            2960 non-null object
Nebenkosten                          2960 non-null object
Heizkosten                           2960 non-null object
Gesamtmiete                          2960 non-null object
Kaution o. Genossenschaftsanteile    2742 non-null object
Etage                                2648 non-null object
Wohnfläche ca.                       2960 non-null object
Zimmer                               2960 non-null object
Schlafzimmer                         1695 non-null float64
Badezimmer                           2155 non-null float64
Personenaufzug                       1551 non-null object
Einbauküche                          1895 non-null object
Baujahr                              2599 non-null object
Modernisierung/ Sanie

# Data Transformation

Showcasing of different methods how data can be manipulated/cleaned up:

- str.replace
- .replace with regex
- loc selection and assignment
- custom functions
- lambda functions

In [111]:
data['Kaltmiete'] = pd.to_numeric(data['Kaltmiete'].str.replace('.','').str.replace(',','.').str.replace(' €',''))

In [112]:
data['Nebenkosten'] = data['Nebenkosten'].str.replace('.','').str.replace(',','.').str.replace(' €','')
data.loc[data['Nebenkosten'] == 'keine Angabe', 'Nebenkosten' ] = np.nan
data['Nebenkosten'] = pd.to_numeric(data['Nebenkosten'])

In [113]:
def transform_heizkosten(x):
    x = x.replace('.','').replace(',','.').replace(' €','')
    if x == 'in Nebenkosten enthalten' or 'inkl' in x:
        x = 0
    elif x == 'nicht in Nebenkosten enthalten' or x == 'keine Angabe':
        x = np.nan
    else:
        x
    return x

data['Heizkosten'] = pd.to_numeric(data['Heizkosten'].apply(transform_heizkosten))

In [114]:
data['Gesamtmiete'] = data['Gesamtmiete'].replace({r" €":"", r"\.":""},regex=True).replace({r"\,":"."},regex=True)
data['Gesamtmiete'] = pd.to_numeric(data['Gesamtmiete'].replace(to_replace=[r" \(zzgl Nebenkosten & Heizkosten\)",
                                                     r" \(zzgl Heizkosten\)",
                                                     r" \(zzgl Nebenkosten\)"],value='',regex=True))

In [115]:
data['Wohnfläche ca.'] = pd.to_numeric(
    data['Wohnfläche ca.'].apply(lambda string: string.replace(' m²','').replace('.','').replace(',','.')))

In [116]:
data['Kaution o. Genossenschaftsanteile'] = data['Kaution o. Genossenschaftsanteile'].replace(to_replace= [r" EUR",r" \€", r"\.",r"\€"],
                                                  value= '',
                                                  regex=True).replace().replace('\,','.',regex=True)

In [117]:
# To be further transformed if necessary
data['Kaution o. Genossenschaftsanteile'].sort_values(ascending=False).head()

2747    zwei Nettokaltmieten
1792                    nein
1323          nach Absprache
2514                entfällt
765     drei Nettokaltmieten
Name: Kaution o. Genossenschaftsanteile, dtype: object

In [118]:
data['Baujahr'] = pd.to_numeric(data['Baujahr'].replace("unbekannt",''))

In [119]:
data['Modernisierung/ Sanierung'] = pd.to_numeric(data['Modernisierung/ Sanierung'].replace(r"zuletzt ",'',regex=True))

In [120]:
data["Energieverbrauchskennwert"] = pd.to_numeric(data["Energieverbrauchskennwert"].replace(r" kWh/\(m\²\*a\)","",regex=True).replace(r"\.","",regex=True).replace(r"\,",".",regex=True))

In [121]:
# Correcting small spelling mistake
data.rename(columns={'Adreese':'Adresse'},inplace=True)

In [122]:
def split_address(x):
    address_list = x.split(',')
    length = len(address_list)
    street = np.nan
    zip_code = np.nan
    kiez = np.nan
    if length == 3:
        street = address_list[0].strip()
        zip_code = address_list[1].strip()
        kiez = address_list[2].strip()
        #return street, zip_code, kiez
    if length == 2:
        street = np.nan
        zip_code = address_list[0].strip()
        kiez = address_list[1].strip()
        kiez = kiez.replace(' Die vollständige Adresse der Immobilie erhalten Sie vom Anbieter.','').strip()
    
    return street, zip_code, kiez

address = pd.DataFrame.from_records(data['Adresse'].apply(split_address)).rename(columns={0:'Straße',1:'PLZ',2:'Kiez'})
data = data.join(address)

In [123]:
def split_street_house_number(string):
    street = np.nan
    number = np.nan
    
    if isinstance(string, str):
        match = re.search('\d\s?\w*',string)
        if match:
            street = string[:match.span()[0]].strip()
            street = street.replace('Str.','Straße').replace('str.','straße')
            number = string[match.span()[0]:match.span()[1]].strip()
    else:
        string
    
    return pd.Series({'Straße':street, 'Housenumber':number})

In [124]:
data[['Straße','Hausnummer']] = data['Straße'].apply(split_street_house_number)

In [125]:
# extract house number only without letter
data['Hausnummer'] = data['Hausnummer'].str.extract(r'(\d*)')

In [126]:
data['Ortsteil'] = data['Kiez'].str.split('(',expand=True)[0].str.strip(')').str.strip()

In [127]:
abbreviations = pd.read_csv('cleaned_abrevations.csv')
abbreviations = abbreviations[abbreviations['Ortsteil'].notnull()]
abbreviations.drop_duplicates('Ortsteil',inplace=True)

In [129]:
data = pd.merge(data, abbreviations, left_on='Ortsteil', right_on='Ortsteil', how='left')

In [130]:
data

,Titel,Url,Kaltmiete,Nebenkosten,Heizkosten,Gesamtmiete,Kaution o. Genossenschaftsanteile,Etage,Wohnfläche ca.,Zimmer,...,Energieverbrauchskennwert,Anbieter,Adresse,Straße,PLZ,Kiez,Hausnummer,Ortsteil,Bezirk,Ortsteilabkürzung
0,Nutzen Sie Ihre Chance - 2 Zimmerwohnung mit b...,https://www.immobilienscout24.de/expose/114414026,979.40,179.00,72.00,1265.40,2938.20,2 von 4,78.00,2,...,114.1,German Prop Tech GmbH,"Reinickendorfer Straße 4, 13447 Berlin, Weddin...",Reinickendorfer Straße,13447 Berlin,Wedding (Wedding),4,Wedding,Mitt,Weddg
1,Komfort 1 Zimmer Apartment - Möbliert in Topci...,https://www.immobilienscout24.de/expose/113507810,895.00,100.00,0.00,995.00,2685,3 von 6,30.97,1,...,NaN,first single apartment verwaltungs GmbH & Co. ...,"Leibnizstraße 85, 10625 Berlin, Charlottenburg...",Leibnizstraße,10625 Berlin,Charlottenburg (Charlottenburg),85,Charlottenburg,ChWi,Charl
2,Sofort bezugsfertig: 2-Zimmer-Wohnung mit Terr...,https://www.immobilienscout24.de/expose/109379660,798.98,197.11,0.00,996.09,2396.93,NaN,67.71,2,...,NaN,allod Immobilien- und Vermögensverwaltungsges....,"Am Hohen Feld 125a , 13125 Berlin, Karow (Weiß...",Am Hohen Feld,13125 Berlin,Karow (Weißensee),125,Karow,Pank,Karow
3,Sofort bezugsfertig: 3-Zimmer-Dachgeschosswohn...,https://www.immobilienscout24.de/expose/111945160,1158.95,257.14,0.00,1416.09,3476.85,NaN,89.15,3,...,NaN,allod Immobilien- und Vermögensverwaltungsges....,"Am Hohen Feld 125a , 13125 Berlin, Karow (Weiß...",Am Hohen Feld,13125 Berlin,Karow (Weißensee),125,Karow,Pank,Karow
4,"*Balkon*, *EBK*, *Erstbezug*, Tolle 3 Zi. Whg.",https://www.immobilienscout24.de/expose/115629249,1447.30,164.00,82.00,1693.30,2895,4 von 5,82.00,3,...,NaN,ADLER Wohnen Service GmbH Gänsemarkt 50 20354 ...,"Hedwig-Porschütz-Straße 2, 10557 Berlin, Tierg...",Hedwig-Porschütz-Straße,10557 Berlin,Tiergarten (Tiergarten),2,Tiergarten,Mitt,Tierg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955,Maisonette-Wohnung in attraktiver Stadtvilla m...,https://www.immobilienscout24.de/expose/43357148,1068.44,125.00,125.00,1318.44,3 Nettokaltmieten,2 von 2,117.04,4,...,NaN,Panita GmbH Möllhausenufer 16 12557 Berlin,"Müggelbergallee 5, 12557 Berlin, Köpenick (Köp...",Müggelbergallee,12557 Berlin,Köpenick (Köpenick),5,Köpenick,TrKö,Köpen
2956,Prenzlauer Berg! 2-Zimmer-Wohnung! Besichtigun...,https://www.immobilienscout24.de/expose/51881358,994.12,109.16,85.77,1189.05,3 Nettokaltmieten,4 von 4,77.97,2,...,NaN,JS-ES GbR,"Meyerheimstr. 15, 10439 Berlin, Prenzlauer Ber...",Meyerheimstraße,10439 Berlin,Prenzlauer Berg (Prenzlauer Berg),15,Prenzlauer Berg,Pank,Prenz
2957,Wohnen am Görlitzer Park,https://www.immobilienscout24.de/expose/46482339,1300.00,200.00,300.00,1800.00,3900,0 von 5,132.00,5,...,NaN,Ohmstr. 1 80802 München,"Görlitzer Str. 43, 10997 Berlin-Kreuzerg, Kreu...",Görlitzer Straße,10997 Berlin-Kreuzerg,Kreuzberg (Kreuzberg),43,Kreuzberg,FrKr,Kreuz
2958,Helle große 3-Zimmerwohnung in Friedrichshain ...,https://www.immobilienscout24.de/expose/53468313,1128.00,95.00,115.00,1338.00,3384,4 von 4,94.00,3,...,NaN,Steinke Albrecht Mitzkait GbR,"Mühsamstraße 66, 10249 Berlin, Friedrichshain ...",Mühsamstraße,10249 Berlin,Friedrichshain (Friedrichshain),66,Friedrichshain,FrKr,Frhai


## Feature Engineering

In [25]:
data['Kaltmiete pro m²'] = data['Kaltmiete']/data['Wohnfläche ca.']

### Location Rating

PDF Source: https://www.stadtentwicklung.berlin.de/wohnen/mietspiegel/de/download/Strassenverzeichnis2019.pdf

In [91]:
# Extracting left tables
top = 69
left = 39
height = 723
width = 253

y1 = top
x1 = left
y2 = top + height
x2 = left + width

area=(y1,x1,y2,x2)
columns = [39,164.23,186.03,193.35,244.96,256.63,286.21]

dfs_1 = tabula.read_pdf("Strassenverzeichnis2019.pdf", 
                     pages='3-262', 
                     area=area,
                     columns=columns,
                     pandas_options={'columns':['Unnamed: 0',
                                                'Straße',
                                                'Bezirk',
                                                'Gebietsstand',
                                                'Hausnummer',
                                                'Buchstabe',
                                               'Wohnlage Einstufung',
                                               'Lage im Stadtgebiet']},
                     guess=False)

# Extracting left tables
top = 69
left = 303
height = 723
width = 253

y1 = top
x1 = left
y2 = top + height
x2 = left + width

area=(y1,x1,y2,x2)
columns = [303,428, 450, 457,509, 520,550]

dfs_2 = tabula.read_pdf("Strassenverzeichnis2019.pdf", 
                     pages='3-262', 
                     area=area,
                     columns=columns,
                     pandas_options={'columns':['Unnamed: 0',
                                                'Straße',
                                                'Bezirk',
                                                'Gebietsstand',
                                                'Hausnummer',
                                                'Buchstabe',
                                               'Wohnlage Einstufung',
                                               'Lage im Stadtgebiet']},
                     guess=False)

dfs_1.append(dfs_2)

In [92]:
# Create new empty final dataframe
location_info = pd.DataFrame()
tables = dfs_1
for table in tables:
    location_info = location_info.append(table)

In [93]:
location_info.drop(columns='Unnamed: 0',inplace=True)

In [94]:
# Drop rows without a bezirk
location_info = location_info[location_info['Bezirk'].notnull()]

# Strip Text
location_info = location_info.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [163]:
def match_street(x):
    if isinstance(x, str):
        match = process.extractOne(x, list(location_info['Straße'].unique()))
        if match[1] > 90:
            return match[0]
    else:
        return x

In [165]:
data['Straße'].head(5).apply(match_street)

0    Reinickendorfer Straße
1             Leibnizstraße
2             Am Hohen Feld
3             Am Hohen Feld
4                      None
Name: Straße, dtype: object

In [160]:
data

,Titel,Url,Kaltmiete,Nebenkosten,Heizkosten,Gesamtmiete,Kaution o. Genossenschaftsanteile,Etage,Wohnfläche ca.,Zimmer,...,Energieverbrauchskennwert,Anbieter,Adresse,Straße,PLZ,Kiez,Hausnummer,Ortsteil,Bezirk,Ortsteilabkürzung
0,Nutzen Sie Ihre Chance - 2 Zimmerwohnung mit b...,https://www.immobilienscout24.de/expose/114414026,979.40,179.00,72.00,1265.40,2938.20,2 von 4,78.00,2,...,114.1,German Prop Tech GmbH,"Reinickendorfer Straße 4, 13447 Berlin, Weddin...",Reinickendorfer Straße,13447 Berlin,Wedding (Wedding),4,Wedding,Mitt,Weddg
1,Komfort 1 Zimmer Apartment - Möbliert in Topci...,https://www.immobilienscout24.de/expose/113507810,895.00,100.00,0.00,995.00,2685,3 von 6,30.97,1,...,NaN,first single apartment verwaltungs GmbH & Co. ...,"Leibnizstraße 85, 10625 Berlin, Charlottenburg...",Leibnizstraße,10625 Berlin,Charlottenburg (Charlottenburg),85,Charlottenburg,ChWi,Charl
2,Sofort bezugsfertig: 2-Zimmer-Wohnung mit Terr...,https://www.immobilienscout24.de/expose/109379660,798.98,197.11,0.00,996.09,2396.93,NaN,67.71,2,...,NaN,allod Immobilien- und Vermögensverwaltungsges....,"Am Hohen Feld 125a , 13125 Berlin, Karow (Weiß...",Am Hohen Feld,13125 Berlin,Karow (Weißensee),125,Karow,Pank,Karow
3,Sofort bezugsfertig: 3-Zimmer-Dachgeschosswohn...,https://www.immobilienscout24.de/expose/111945160,1158.95,257.14,0.00,1416.09,3476.85,NaN,89.15,3,...,NaN,allod Immobilien- und Vermögensverwaltungsges....,"Am Hohen Feld 125a , 13125 Berlin, Karow (Weiß...",Am Hohen Feld,13125 Berlin,Karow (Weißensee),125,Karow,Pank,Karow
4,"*Balkon*, *EBK*, *Erstbezug*, Tolle 3 Zi. Whg.",https://www.immobilienscout24.de/expose/115629249,1447.30,164.00,82.00,1693.30,2895,4 von 5,82.00,3,...,NaN,ADLER Wohnen Service GmbH Gänsemarkt 50 20354 ...,"Hedwig-Porschütz-Straße 2, 10557 Berlin, Tierg...",Hedwig-Porschütz-Straße,10557 Berlin,Tiergarten (Tiergarten),2,Tiergarten,Mitt,Tierg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955,Maisonette-Wohnung in attraktiver Stadtvilla m...,https://www.immobilienscout24.de/expose/43357148,1068.44,125.00,125.00,1318.44,3 Nettokaltmieten,2 von 2,117.04,4,...,NaN,Panita GmbH Möllhausenufer 16 12557 Berlin,"Müggelbergallee 5, 12557 Berlin, Köpenick (Köp...",Müggelbergallee,12557 Berlin,Köpenick (Köpenick),5,Köpenick,TrKö,Köpen
2956,Prenzlauer Berg! 2-Zimmer-Wohnung! Besichtigun...,https://www.immobilienscout24.de/expose/51881358,994.12,109.16,85.77,1189.05,3 Nettokaltmieten,4 von 4,77.97,2,...,NaN,JS-ES GbR,"Meyerheimstr. 15, 10439 Berlin, Prenzlauer Ber...",Meyerheimstraße,10439 Berlin,Prenzlauer Berg (Prenzlauer Berg),15,Prenzlauer Berg,Pank,Prenz
2957,Wohnen am Görlitzer Park,https://www.immobilienscout24.de/expose/46482339,1300.00,200.00,300.00,1800.00,3900,0 von 5,132.00,5,...,NaN,Ohmstr. 1 80802 München,"Görlitzer Str. 43, 10997 Berlin-Kreuzerg, Kreu...",Görlitzer Straße,10997 Berlin-Kreuzerg,Kreuzberg (Kreuzberg),43,Kreuzberg,FrKr,Kreuz
2958,Helle große 3-Zimmerwohnung in Friedrichshain ...,https://www.immobilienscout24.de/expose/53468313,1128.00,95.00,115.00,1338.00,3384,4 von 4,94.00,3,...,NaN,Steinke Albrecht Mitzkait GbR,"Mühsamstraße 66, 10249 Berlin, Friedrichshain ...",Mühsamstraße,10249 Berlin,Friedrichshain (Friedrichshain),66,Friedrichshain,FrKr,Frhai


In [134]:
test = pd.merge(data, location_info.drop_duplicates('Straße')[['Straße','Buchstabe']],left_on='Straße',right_on='Straße',how='left')

In [30]:
def transform_housenumbers(x):
    if isinstance(x, str):
        start = x.split('-')[0].strip()
        end = x.split('-')[1].strip()
        
        start = int(re.search('\d*', start).group())
        end = int(re.search('\d*', end).group())
        return list(range(start,end+1))
    else:
        return x      

In [31]:
location_info['hausnummer'] = location_info['hausnummer'].apply(transform_housenumbers)

In [32]:
# Add bezirk details
bezirk_abbreviations = abbreviations
bezirk_abbreviations.columns = bezirk_abbreviations.columns.str.lower()
bezirk_abbreviations = bezirk_abbreviations[bezirk_abbreviations['ortsteilabkürzung'].isnull()].drop(46).drop(columns='ortsteilabkürzung')

# Add full name of bezirk to df
location_info = pd.merge(location_info, bezirk_abbreviations, how='left')

In [33]:
location_info = location_info.explode('hausnummer')

In [45]:
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
location_info = location_info.applymap(lambda x: x.strip() if isinstance(x, str) else x)

### Merge location rating to scraped data

In [57]:
data['Hausnummer'] = pd.to_numeric(data['Hausnummer'])

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2960 entries, 0 to 2959
Data columns (total 32 columns):
Titel                                2960 non-null object
Url                                  2960 non-null object
Kaltmiete                            2960 non-null float64
Nebenkosten                          2887 non-null float64
Heizkosten                           2610 non-null float64
Gesamtmiete                          2960 non-null float64
Kaution o. Genossenschaftsanteile    2960 non-null object
Etage                                2648 non-null object
Wohnfläche ca.                       2960 non-null float64
Zimmer                               2960 non-null object
Schlafzimmer                         1695 non-null float64
Badezimmer                           2155 non-null float64
Personenaufzug                       1551 non-null object
Einbauküche                          1895 non-null object
Baujahr                              2524 non-null float64
Modernisierung/

In [59]:
data.head(1)

,Titel,Url,Kaltmiete,Nebenkosten,Heizkosten,Gesamtmiete,Kaution o. Genossenschaftsanteile,Etage,Wohnfläche ca.,Zimmer,...,Anbieter,Adresse,Straße,PLZ,Kiez,Hausnummer,Ortsteil,Bezirk,Ortsteilabkürzung,Kaltmiete pro m²
0,Nutzen Sie Ihre Chance - 2 Zimmerwohnung mit b...,https://www.immobilienscout24.de/expose/114414026,979.4,179.0,72.0,1265.4,2938.20,2 von 4,78.0,2,...,German Prop Tech GmbH,"Reinickendorfer Straße 4, 13447 Berlin, Weddin...",Reinickendorfer Straße,13447 Berlin,Wedding (Wedding),4.0,Wedding,Mitt,Weddg,12.55641


In [75]:
location_info.groupby(['straße','bezirk','wohnlage_einstufung']).count().to_csv('check.csv')

In [61]:
test = pd.merge(data, location_info, left_on=['Straße','Bezirk','Hausnummer'], right_on=['straße','bezirk','hausnummer'], how='left')

In [80]:
test[['Adresse','Straße','wohnlage_einstufung','buchstabe']]

,Adresse,Straße,wohnlage_einstufung,buchstabe
0,"Reinickendorfer Straße 4, 13447 Berlin, Weddin...",Reinickendorfer Straße,einfach *,F
1,"Leibnizstraße 85, 10625 Berlin, Charlottenburg...",Leibnizstraße,gut *,F
2,"Am Hohen Feld 125a , 13125 Berlin, Karow (Weiß...",Am Hohen Feld,NaN,NaN
3,"Am Hohen Feld 125a , 13125 Berlin, Karow (Weiß...",Am Hohen Feld,NaN,NaN
4,"Hedwig-Porschütz-Straße 2, 10557 Berlin, Tierg...",Hedwig-Porschütz-Straße,NaN,NaN
...,...,...,...,...
3286,"Mühsamstraße 66, 10249 Berlin, Friedrichshain ...",Mühsamstraße,mittel,G
3287,"Stresowplatz 16a, 13597 Berlin, Spandau (Spandau)",Stresowplatz,einfach,F
3288,"Stresowplatz 16a, 13597 Berlin, Spandau (Spandau)",Stresowplatz,einfach *,F
3289,"Stresowplatz 16a, 13597 Berlin, Spandau (Spandau)",Stresowplatz,einfach,F


In [90]:
test[test['wohnlage_einstufung'].isnull()]

,Titel,Url,Kaltmiete,Nebenkosten,Heizkosten,Gesamtmiete,Kaution o. Genossenschaftsanteile,Etage,Wohnfläche ca.,Zimmer,...,Ortsteilabkürzung,Kaltmiete pro m²,straße,bezirk,gebietsstand,hausnummer,buchstabe,wohnlage_einstufung,lage_im_stadtgebiet,ortsteil
2,Sofort bezugsfertig: 2-Zimmer-Wohnung mit Terr...,https://www.immobilienscout24.de/expose/109379660,798.98,197.11,0.00,996.09,2396.93,NaN,67.71,2,...,Karow,11.800030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sofort bezugsfertig: 3-Zimmer-Dachgeschosswohn...,https://www.immobilienscout24.de/expose/111945160,1158.95,257.14,0.00,1416.09,3476.85,NaN,89.15,3,...,Karow,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"*Balkon*, *EBK*, *Erstbezug*, Tolle 3 Zi. Whg.",https://www.immobilienscout24.de/expose/115629249,1447.30,164.00,82.00,1693.30,2895,4 von 5,82.00,3,...,Tierg,17.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Eine Perle mit modernem Grundriss,https://www.immobilienscout24.de/expose/115502827,1290.00,210.00,140.00,1640.00,3 NKM,5 von 1,91.89,3,...,Frhai,14.038524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,KOMFORTABEL WOHNEN IM ERSTBEZUG,https://www.immobilienscout24.de/expose/115632638,799.00,116.00,87.00,1002.00,3 Nettokaltmieten Kaution,2 von 7,72.56,3,...,Helld,11.011577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,Dachgeschosswohnung in Tiergarten: Individuell...,https://www.immobilienscout24.de/expose/38427207,1980.00,140.00,140.00,2260.00,5940.00,5 von 5,161.24,3,...,Tierg,12.279831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3276,3 Zimmerwohnung WG-geeignet in schöneberger To...,https://www.immobilienscout24.de/expose/54267836,1015.00,284.00,0.00,1299.00,3045,2,73.00,3,...,Schön,13.904110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3280,"**Super schönes helles Loft mit 5 Zimmern, Dac...",https://www.immobilienscout24.de/expose/42687441,2860.00,240.00,0.00,3360.00,8580.00,5 von 5,260.00,5,...,Frhai,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3283,Prenzlauer Berg! 2-Zimmer-Wohnung! Besichtigun...,https://www.immobilienscout24.de/expose/51881358,994.12,109.16,85.77,1189.05,3 Nettokaltmieten,4 von 4,77.97,2,...,Prenz,12.750032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
"""
Erstmalige Bezugsfertigkeit der Wohnung und Ausstattung	Mietpreis pro Quadratmeter
bis 1918 mit Sammelheizung und mit Bad	6,45 Euro
bis 1918 mit Sammelheizung oder mit Bad	5,00 Euro
bis 1918 ohne Sammelheizung und ohne Bad	3,92 Euro
1919 bis 1949 mit Sammelheizung und mit Bad	6,27 Euro
1919 bis 1949 mit Sammelheizung oder mit Bad	5,22 Euro
1919 bis 1949 ohne Sammelheizung und ohne Bad	4,59 Euro
1950 bis 1964 mit Sammelheizung und mit Bad	6,08 Euro
1950 bis 1964 mit Sammelheizung oder mit Bad	5,62 Euro
1965 bis 1972 mit Sammelheizung und mit Bad	5,95 Euro
1973 bis 1990 mit Sammelheizung und mit Bad	6,04 Euro
1991 bis 2002 mit Sammelheizung und mit Bad	8,13 Euro
2003 bis 2013 mit Sammelheizung und mit Bad	9,80 Euro
"""

In [ ]:
rental_ceiling_info = {
    'min_year': [1918, 1918, 1918, 1919, 1919, 1919, 1950, 1950, 1965, 1973, 1991, 2003 ],
    'max_year': [1918, 1918, 1918, 1949, 1949, 1949, 1964, 1964, 1972, 1990, 2002, 2013 ],
    'sammelheizung' : [True, True, False, True, True, False, True, True, True, True, True, True],
    'bad' : [True, True, False, True, True, False, True, True, True, True, True, True],
    'condition': ['and', 'or', 'and', 'and', 'or', 'and', 'and', 'or', 'and', 'and', 'and', 'and'],
    'price_per_square_meter' : [6.45, 5.00, 3.92, 6.27, 5.22, 4.59, 6.08, 5.62, 5.95, 6.04, 8.13, 9.8]
}

In [ ]:
len(rental_ceiling_info['price_per_square_meter'])

In [ ]:
rental_ceiling_info = pd.DataFrame.from_dict(rental_ceiling_info)

In [ ]:
rental_ceiling_info

In [ ]:
def find_price_ceiling(df):
    if df['Baujahr'] <= 1918:
        max_price = 6.45
    elif df['Baujahr'] >= 1919 and df['Baujahr'] <= 1949:
        max_price = 6.27
    elif df['Baujahr'] >= 1950 and df['Baujahr'] <= 1964:
        max_price = 6.08
    elif df['Baujahr'] >= 1965 and df['Baujahr'] <= 1972:
        max_price = 5.95
    elif df['Baujahr'] >= 1973 and df['Baujahr'] <= 1990:
        max_price = 6.04
    elif df['Baujahr'] >= 1991 and df['Baujahr'] <= 2002:
        max_price = 8.13
    elif df['Baujahr'] >= 2003 and df['Baujahr'] <= 2013:
        max_price = 9.80
    else:
        max_price = np.nan
    
    return max_price

In [ ]:
data['Mietendeckel pro Quadratmeter ohne Modernisierung']=  data.apply(find_price_ceiling,axis=1) * 1.2

In [ ]:
illegal = data[data['Kaltmiete pro m²'] > data['Mietendeckel pro Quadratmeter ohne Modernisierung']]

In [ ]:
illegal.groupby('Anbieter').count().sort_values(by='Titel',ascending=False)

In [ ]:
not_illegal = data[data['Kaltmiete pro m²'] < data['Mietendeckel pro Quadratmeter ohne Modernisierung']]

In [ ]:
not_illegal.groupby('Anbieter').count().sort_values(by='Titel',ascending=False)

In [ ]:
data